# 投资组合优化问题

假定我现在需要进行一次投资，我的初始投资额是一定的，但是每次做决策时对应的利润率会改变，我该如何决定我的投资？

这个模型具体描述如下：

目标函数
1. 目标函数是最大化各种情况下的利润(需要参数来设置收益系数和损失厌恶系数)

约束条件：
1. 初始投资额约束，第一年的投资=初始金额
2. 状态转移约束，上一时刻末的投资额 = 下一时刻初的投资额
3. 最终收益约束，最后一年的投资额 = 最终收益

$$
\begin{aligned}
& \max z=\sum_s p(s)(q y(s)-r w(s)) \\
& \text { s. t. } \sum_{i=1}^I x(i, 1, s)=b, \forall s \in S \text {, } \\
& \sum_{i=1}^I \xi(i, t, s) x(i, t-1, s)-\sum_{i=1}^I x(i, t, s)=0, \forall s \in S, \\
& t=2, \ldots, H, \\
& \sum_{i=1}^I \xi(i, H, s) x(i, H, s)-y(s)+w(s)=G, \\
& \left(\sum_{s^{\prime} \in S_{J(s, t)}^{\prime}} p\left(s^{\prime}\right) x\left(i, t, s^{\prime}\right)\right)-\left(\sum_{s^{\prime} \in S_{J(s, t)}^t} p\left(s^{\prime}\right)\right) x(i, t, s)=0, \\
& \forall 1 \leq i \leq I, \forall 1 \leq t \leq H, \forall s \in S, \\
& x(i, t, s) \geq 0, y(s) \geq 0, w(s) \geq 0, \\
& \forall 1 \leq i \leq I, \forall 1 \leq t \leq H, \forall s \in S,
\end{aligned}
$$

In [12]:
import gurobipy as gp
from gurobipy import GRB

# 创建模型
model = gp.Model("portfolio_investment")

# 定义变量
# 变量 x(i, t) 表示在时间 t 做出第 i 次决策
x_it = model.addVars([(i, t) for i in range(1, 3) for t in range(1, 2)],lb=0, vtype=GRB.CONTINUOUS, name="x_it")

# 变量 x(i, t, s1) 表示第 i 次决策，在时间 t 的状态下，上一决策情况为 s1
x_its1 = model.addVars([(i, t, s1) for i in range(1, 3) for t in range(1, 3) for s1 in range(1, 3)],lb=0, vtype=GRB.CONTINUOUS, name="x_its1")

# 变量 x(i, t, s1, s2) 表示第 i 次决策，在时间 t 的状态下，决策情况为 s1 和 s2
x_its1s2 = model.addVars([(i, t, s1, s2) for i in range(1,3) for t in range(1, 4) for s1 in range(1, 3) for s2 in range(1, 3)],lb=0, vtype=GRB.CONTINUOUS, name="x_its1s2")

y = model.addVars([(s1, s2, s3) for s1 in range(1, 3) for s2 in range(1, 3) for s3 in range(1, 3)], lb=0, vtype=GRB.CONTINUOUS, name="y")
w = model.addVars([(s1, s2, s3) for s1 in range(1, 3) for s2 in range(1, 3) for s3 in range(1, 3)], lb=0, vtype=GRB.CONTINUOUS, name="w")

# 设置目标函数
model.setObjective(
    gp.quicksum(0.125 * (y[s1, s2, s3] - 4 * w[s1, s2, s3]) for s1 in range(1, 3) for s2 in range(1, 3) for s3 in range(1, 3)),
    GRB.MAXIMIZE
)

# 添加约束
model.addConstr(x_it[1, 1] + x_it[2, 1] == 55, name="constr_1")

# 状态转移约束 - t=1 和 t=2 时刻的投资额相同
model.addConstr(-1.25 * x_it[1, 1] - 1.14 * x_it[2, 1] + x_its1[1, 2, 1] + x_its1[2, 2, 1] == 0, name="constr_2")
model.addConstr(-1.06 * x_it[1, 1] - 1.12 * x_it[2, 1] + x_its1[1, 2, 2] + x_its1[2, 2, 2] == 0, name="constr_2_2")

# 状态转移约束 - t=2 和 t=3 时刻的投资额相同
model.addConstr(-1.25 * x_its1[1, 2, 1] - 1.14 * x_its1[2, 2, 1] + x_its1s2[1, 3, 1, 1] + x_its1s2[2, 3, 1, 1] == 0, name="constr_3")
model.addConstr(-1.06 * x_its1[1, 2, 1] - 1.12 * x_its1[2, 2, 1] + x_its1s2[1, 3, 1, 2] + x_its1s2[2, 3, 1, 2] == 0, name="constr_3_2")

# 状态转移约束 - t=2 和 t=3 时刻的投资额相同
model.addConstr(-1.25 * x_its1[1, 2, 2] - 1.14 * x_its1[2, 2, 2] + x_its1s2[1, 3, 2, 1] + x_its1s2[2, 3, 2, 1] == 0, name="constr_4")
model.addConstr(-1.06 * x_its1[1, 2, 2] - 1.12 * x_its1[2, 2, 2] + x_its1s2[1, 3, 2, 2] + x_its1s2[2, 3, 2, 2] == 0, name="constr_4_2")


# 不同选择下最终收益要为80
model.addConstr(1.25 * x_its1s2[1, 3, 1, 1] + 1.14 * x_its1s2[2, 3, 1, 1] - y[1, 1, 1] + w[1, 1, 1] == 80, name="constr_6")
model.addConstr(1.06 * x_its1s2[1, 3, 1, 1] + 1.12 * x_its1s2[2, 3, 1, 1] - y[1, 1, 2] + w[1, 1, 2] == 80, name="constr_7")
model.addConstr(1.25 * x_its1s2[1, 3, 1, 2] + 1.14 * x_its1s2[2, 3, 1, 2] - y[1, 2, 1] + w[1, 2, 1] == 80, name="constr_8")
model.addConstr(1.06 * x_its1s2[1, 3, 1, 2] + 1.12 * x_its1s2[2, 3, 1, 2] - y[1, 2, 2] + w[1, 2, 2] == 80, name="constr_9")
model.addConstr(1.25 * x_its1s2[1, 3, 2, 1] + 1.14 * x_its1s2[2, 3, 2, 1] - y[2, 1, 1] + w[2, 1, 1] == 80, name="constr_10")
model.addConstr(1.06 * x_its1s2[1, 3, 2, 1] + 1.12 * x_its1s2[2, 3, 2, 1] - y[2, 1, 2] + w[2, 1, 2] == 80, name="constr_11")
model.addConstr(1.25 * x_its1s2[1, 3, 2, 2] + 1.14 * x_its1s2[2, 3, 2, 2] - y[2, 2, 1] + w[2, 2, 1] == 80, name="constr_12")
model.addConstr(1.06 * x_its1s2[1, 3, 2, 2] + 1.12 * x_its1s2[2, 3, 2, 2] - y[2, 2, 2] + w[2, 2, 2] == 80, name="constr_13")

# 优化模型
model.setParam("OutputFlag", 0)
model.optimize()

# 输出结果
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.objVal)
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName} = {v.x}")
else:
    print("No optimal solution found.")

Optimal objective value: -1.5140846428571102
x_it[1,1] = 41.47927229346878
x_it[2,1] = 13.52072770653122
x_its1[1,2,1] = 65.09458196639906
x_its1[1,2,2] = 36.74321503131523
x_its1[2,2,1] = 2.1681379858825087
x_its1[2,2,2] = 22.368028631076648
x_its1s2[1,3,1,1] = 83.83990476190489
x_its1s2[1,3,2,2] = 64.0
x_its1s2[2,3,1,2] = 71.42857142857142
x_its1s2[2,3,2,1] = 71.42857142857142
y[1,1,1] = 24.799880952381116
y[1,1,2] = 8.870299047619184
y[1,2,1] = 1.4285714285714022
y[2,1,1] = 1.4285714285714022
w[2,2,2] = 12.159999999999997


## VSS引入

我们可以使用均值来得到Stock 比 Bond的钱更多，Stock的年收益为1.155，Boud为1.13。在这种情况下，大家自然就会选择Stock，那么我们可以计算出在这个选择之下的目标函数值，就可以得到这个信息的价值有多大。

In [23]:
import gurobipy as gp
from gurobipy import GRB

# 创建模型
model = gp.Model("portfolio_investment")

# 定义变量
# 变量 x(i, t) 表示在时间 t 做出第 i 次决策
x_it = model.addVars([(i, t) for i in range(1, 3) for t in range(1, 2)],lb=0, vtype=GRB.CONTINUOUS, name="x_it")

# 变量 x(i, t, s1) 表示第 i 次决策，在时间 t 的状态下，上一决策情况为 s1
x_its1 = model.addVars([(i, t, s1) for i in range(1, 3) for t in range(1, 3) for s1 in range(1, 3)],lb=0, vtype=GRB.CONTINUOUS, name="x_its1")

# 变量 x(i, t, s1, s2) 表示第 i 次决策，在时间 t 的状态下，决策情况为 s1 和 s2
x_its1s2 = model.addVars([(i, t, s1, s2) for i in range(1,3) for t in range(1, 4) for s1 in range(1, 3) for s2 in range(1, 3)],lb=0, vtype=GRB.CONTINUOUS, name="x_its1s2")

y = model.addVars([(s1, s2, s3) for s1 in range(1, 3) for s2 in range(1, 3) for s3 in range(1, 3)], lb=0, vtype=GRB.CONTINUOUS, name="y")
w = model.addVars([(s1, s2, s3) for s1 in range(1, 3) for s2 in range(1, 3) for s3 in range(1, 3)], lb=0, vtype=GRB.CONTINUOUS, name="w")

# 设置目标函数
model.setObjective(
    gp.quicksum(0.125 * (y[s1, s2, s3] - 4 * w[s1, s2, s3]) for s1 in range(1, 3) for s2 in range(1, 3) for s3 in range(1, 3)),
    GRB.MAXIMIZE
)

# 添加约束

# 让所有选择都选择stock，因为stock的EV >= bond的EV
model.addConstr(x_it[2,1]==0)
model.addConstr(x_its1[2,2,1]==0)
model.addConstr(x_its1[2,2,2]==0)
model.addConstr(x_its1s2[2,3,1,1]==0)
model.addConstr(x_its1s2[2,3,1,2]==0)
model.addConstr(x_its1s2[2,3,2,1]==0)
model.addConstr(x_its1s2[2,3,2,2]==0)


model.addConstr(x_it[1, 1] + x_it[2, 1] == 55, name="constr_1")

# 状态转移约束 - t=1 和 t=2 时刻的投资额相同
model.addConstr(-1.25 * x_it[1, 1] - 1.14 * x_it[2, 1] + x_its1[1, 2, 1] + x_its1[2, 2, 1] == 0, name="constr_2")
model.addConstr(-1.06 * x_it[1, 1] - 1.12 * x_it[2, 1] + x_its1[1, 2, 2] + x_its1[2, 2, 2] == 0, name="constr_2_2")

# 状态转移约束 - t=2 和 t=3 时刻的投资额相同
model.addConstr(-1.25 * x_its1[1, 2, 1] - 1.14 * x_its1[2, 2, 1] + x_its1s2[1, 3, 1, 1] + x_its1s2[2, 3, 1, 1] == 0, name="constr_3")
model.addConstr(-1.06 * x_its1[1, 2, 1] - 1.12 * x_its1[2, 2, 1] + x_its1s2[1, 3, 1, 2] + x_its1s2[2, 3, 1, 2] == 0, name="constr_3_2")

# 状态转移约束 - t=2 和 t=3 时刻的投资额相同
model.addConstr(-1.25 * x_its1[1, 2, 2] - 1.14 * x_its1[2, 2, 2] + x_its1s2[1, 3, 2, 1] + x_its1s2[2, 3, 2, 1] == 0, name="constr_4")
model.addConstr(-1.06 * x_its1[1, 2, 2] - 1.12 * x_its1[2, 2, 2] + x_its1s2[1, 3, 2, 2] + x_its1s2[2, 3, 2, 2] == 0, name="constr_4_2")


# 不同选择下最终收益要为80
model.addConstr(1.25 * x_its1s2[1, 3, 1, 1] + 1.14 * x_its1s2[2, 3, 1, 1] - y[1, 1, 1] + w[1, 1, 1] == 80, name="constr_6")
model.addConstr(1.06 * x_its1s2[1, 3, 1, 1] + 1.12 * x_its1s2[2, 3, 1, 1] - y[1, 1, 2] + w[1, 1, 2] == 80, name="constr_7")
model.addConstr(1.25 * x_its1s2[1, 3, 1, 2] + 1.14 * x_its1s2[2, 3, 1, 2] - y[1, 2, 1] + w[1, 2, 1] == 80, name="constr_8")
model.addConstr(1.06 * x_its1s2[1, 3, 1, 2] + 1.12 * x_its1s2[2, 3, 1, 2] - y[1, 2, 2] + w[1, 2, 2] == 80, name="constr_9")
model.addConstr(1.25 * x_its1s2[1, 3, 2, 1] + 1.14 * x_its1s2[2, 3, 2, 1] - y[2, 1, 1] + w[2, 1, 1] == 80, name="constr_10")
model.addConstr(1.06 * x_its1s2[1, 3, 2, 1] + 1.12 * x_its1s2[2, 3, 2, 1] - y[2, 1, 2] + w[2, 1, 2] == 80, name="constr_11")
model.addConstr(1.25 * x_its1s2[1, 3, 2, 2] + 1.14 * x_its1s2[2, 3, 2, 2] - y[2, 2, 1] + w[2, 2, 1] == 80, name="constr_12")
model.addConstr(1.06 * x_its1s2[1, 3, 2, 2] + 1.12 * x_its1s2[2, 3, 2, 2] - y[2, 2, 2] + w[2, 2, 2] == 80, name="constr_13")


# 优化模型
model.setParam("OutputFlag", 0)
model.optimize()

# 输出结果
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.objVal)
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName} = {v.x}")
else:
    print("No optimal solution found.")
    

Optimal objective value: -3.78791937499998
x_it[1,1] = 55.0
x_its1[1,2,1] = 68.75
x_its1[1,2,2] = 58.300000000000004
x_its1s2[1,3,1,1] = 85.9375
x_its1s2[1,3,1,2] = 72.875
x_its1s2[1,3,2,1] = 72.875
x_its1s2[1,3,2,2] = 61.79800000000001
y[1,1,1] = 27.421875
y[1,1,2] = 11.09375
y[1,2,1] = 11.09375
y[2,1,1] = 11.09375
w[1,2,2] = 2.7524999999999977
w[2,1,2] = 2.7524999999999977
w[2,2,1] = 2.7524999999999835
w[2,2,2] = 14.494119999999981


## VSS计算

VSS = RP - EV = -1.514 -(-3.788) = 2.274